In [ ]:
import pandas as pd
import numpy as np
import time
import os
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# --- [ 0. 경로 및 변수 설정 ] ---
print("--- [ 'One Universe' (Lag + sin/cos + 튜닝 V2) 파이프라인 시작 ] ---")
start_time = time.time()

# --- 원본 데이터 경로 (상대 경로) ---
CSV_TRAIN_PATH = 'train.csv'
CSV_TEST_PATH = 'test.csv'
SAMPLE_SUB_PATH = 'submission_sample.csv'

# --- '스마트' 캐시 경로 (상대 경로) ---
FEATHER_TRAIN_PATH = 'train_original.feather'
FEATHER_TEST_PATH = 'test_original.feather'

# --- 최종 제출 파일 경로 (상대 경로) ---
SUB_PATH = 'submission_Tuned_v2.csv' # 새 제출 파일 이름

# --- 'Y 분리' 대상 컬럼 ---
TARGET_COLS_TO_SEPARATE = ['nins', 'energy']

# --- '단일 보간' 대상 컬럼 (91.6% NaN 컬럼들) ---
COLS_TO_INTERPOLATE = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 
    'pressure', 'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 
    'temp_b', 'uv_idx', 'vis', 'wind_chill_temp', 'wind_dir_b', 
    'wind_gust_spd', 'wind_spd_b', 'cloud_a', 'ground_press', 
    'humidity', 'rain', 'snow', 'temp_a', 'temp_max', 'temp_min', 
    'wind_dir_a', 'wind_spd_a'
]

# --- '단일 군집' 대상 컬럼 ---
CLUSTER_FEATURES = ['coord1', 'coord2']
N_CLUSTERS = 20


try:
    # --- [ 1-3 단계: 스마트 로드, Y 분리, X 결합 ] ---
    print("\n--- [ 1-3 단계: 스마트 로드, Y 분리, X 결합 ] ---")
    if os.path.exists(FEATHER_TRAIN_PATH):
        df_train = pd.read_feather(FEATHER_TRAIN_PATH)
    else:
        df_train = pd.read_csv(CSV_TRAIN_PATH, parse_dates=['time'])
        df_train.to_feather(FEATHER_TRAIN_PATH)
    if os.path.exists(FEATHER_TEST_PATH):
        df_test = pd.read_feather(FEATHER_TEST_PATH)
    else:
        df_test = pd.read_csv(CSV_TEST_PATH, parse_dates=['time'])
        df_test.to_feather(FEATHER_TEST_PATH)

    print("--- [ 2. Y 분리 ] ---")
    y_train = df_train['nins'].copy()
    cols_to_drop_train = [col for col in TARGET_COLS_TO_SEPARATE if col in df_train.columns]
    df_train = df_train.drop(columns=cols_to_drop_train)
    cols_to_drop_test = [col for col in TARGET_COLS_TO_SEPARATE if col in df_test.columns]
    if cols_to_drop_test:
         df_test = df_test.drop(columns=cols_to_drop_test)
    print(f"'y_train' 백업 및 {TARGET_COLS_TO_SEPARATE} 제거 완료.")

    print("--- [ 3. X 결합 (A/B 세계 통일) ] ---")
    df_full = pd.concat([df_train, df_test], ignore_index=True)
    print(f"'df_full' (One Universe) 생성 완료. (총 {len(df_full)} 행)")


    # --- [ 4 단계: '단일' 보간 (ffill/bfill) ] ---
    print("\n--- [ 4 단계: '단일' 보간 (ffill/bfill) ] ---")
    cols_exist = [col for col in COLS_TO_INTERPOLATE if col in df_full.columns]
    df_full[cols_exist] = df_full.groupby('pv_id')[cols_exist].ffill().bfill()
    nan_check = df_full[cols_exist].isna().sum().sum()
    print(f"보간 완료. 남은 NaN 개수: {nan_check}")


    # --- [ 5 단계: '단일' 군집 ] ---
    print("\n--- [ 5 단계: '단일' Scaler + KMeans 군집 ] ---")
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_full[CLUSTER_FEATURES])
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(scaled_features)
    df_full['cluster_id'] = cluster_labels
    print("'cluster_id' 생성 완료.")

    
    # --- [ 5.5 단계 (1순위): Lag 피처 엔지니어링 ] ---
    print("\n--- [ 5.5 단계 (1순위): Lag 피처 생성 ] ---")
    lag_start_time = time.time()
    for col in cols_exist: 
        df_full[f'{col}_lag30m'] = df_full.groupby('pv_id')[col].shift(6)
        df_full[f'{col}_lag1h'] = df_full.groupby('pv_id')[col].shift(12)
    print("Lag 피처 생성 완료. (52개 신규 생성)")
    df_full = df_full.groupby('pv_id').bfill() 
    print(f"Lag 피처 단계 완료. (소요 시간: {time.time() - lag_start_time:.2f} 초)")

    
    # --- [ 5.6 단계 ('정제된' 2순위): '시간' 피처 생성 ] ---
    print("\n--- [ 5.6 단계 ('정제된' 2순위): '시간' 피처 생성 ] ---")
    df_full['hour'] = df_full['time'].dt.hour
    df_full['day_of_year'] = df_full['time'].dt.dayofyear
    df_full['month'] = df_full['time'].dt.month
    df_full['weekday'] = df_full['time'].dt.weekday
    df_full['hour_sin'] = np.sin(2 * np.pi * df_full['hour'] / 24.0)
    df_full['hour_cos'] = np.cos(2 * np.pi * df_full['hour'] / 24.0)
    print("'hour_sin', 'hour_cos' 피처 생성 완료.")


    # --- [ 6 단계: A(Train) / B(Test) 분리 ] ---
    print("\n--- [ 6 단계: A(Train) / B(Test) 분리 ] ---")
    X_train_processed = df_full[df_full['type'] == 'train'].copy()
    X_test_processed = df_full[df_full['type'] == 'test'].copy()
    print(f"X_train: {X_train_processed.shape}, X_test: {X_test_processed.shape}")

    
    # --- [ 7 단계: 최종 피처 정의 ] ---
    print("\n--- [ 7 단계: 최종 피처 리스트 정의 ] ---")
    FEATURES = list(X_train_processed.select_dtypes(include=np.number).columns)
    COLS_TO_DROP_FINAL = ['nins', 'energy'] 
    FEATURES = [col for col in FEATURES if col not in COLS_TO_DROP_FINAL]
    print(f"총 {len(FEATURES)}개의 피처로 학습을 시작합니다.")


    # --- [ 8 단계: 모델 학습 (LightGBM) - (튜닝 V2) ] ---
    print("\n--- [ 8 단계: 모델 학습 (LightGBM) - (튜닝 V2) ] ---")
    
    # [ 튜닝 V2: 127 + 과적합 방지 3종 ]
    lgb_params = {
        'objective': 'mae',
        'metric': 'mae',
        'n_estimators': 10000,          # 조기 종료를 위해 넉넉하게
        'learning_rate': 0.01,         # 53점 모델과 동일 (정밀하게)
        
        # [수정됨] 1. 더 복잡한 모델
        'num_leaves': 127,             # 63 -> 127
        
        # [신규] 2. 과적합 방지 (암기 방지)
        'subsample': 0.8,              # 80%의 행만 무작위로 사용
        'colsample_bytree': 0.7,       # 70%의 피처만 무작위로 사용
        'min_data_in_leaf': 50,        # 리프 노드의 최소 데이터 수
        
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1,
    }

    model = lgb.LGBMRegressor(**lgb_params)
    
    print(f"튜닝 V2 모델 (lr={lgb_params['learning_rate']}, leaves={lgb_params['num_leaves']}) 학습 시작...")
    model.fit(X_train_processed[FEATURES], y_train,
              eval_set=[(X_train_processed[FEATURES], y_train)], # 자체 검증
              callbacks=[lgb.early_stopping(100, verbose=100)]) # 조기 종료


    # --- [ 9 단계: 예측 (Prediction) - (정제됨) ] ---
    print("\n--- [ 9 단계: 예측 ] ---")
    
    predictions = model.predict(X_test_processed[FEATURES])
    
    # 후처리 1: 발전량은 0 미만이 될 수 없음 (유일한 후처리)
    predictions[predictions < 0] = 0
    print("예측 완료: (1) 0 미만 값 0 처리 완료.")


    # --- [ 10 단계: 제출 파일 생성 ] ---
    print("\n--- [ 10 단계: 제출 파일 생성 ] ---")
    
    df_submission = pd.read_csv(SAMPLE_SUB_PATH)
    df_submission['nins'] = predictions
    
    df_submission.to_csv(SUB_PATH, index=False)
    
    end_time = time.time()
    print("\n\n--- [ 'One Universe' (튜닝 V2) 완료 ] ---")
    print(f"최종 제출 파일이 '{SUB_PATH}'에 저장되었습니다.")
    print(f"총 소요 시간: {end_time - start_time:.2f} 초")
    
except FileNotFoundError as e:
    print(f"[치명적 오류] 필수 파일을 찾을 수 없습니다: {e.filename}")
except MemoryError:
    print("[오류] 메모리 부족 (MemoryError).")
except Exception as e:
    print(f"[오류] 파이프라인 중 예기치 않은 문제가 발생했습니다: {e}")

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

# --- 파일 경로 ---
TRUE_FILE = 'true.csv'                  # 정답 파일
SUB_FILE = 'submission_Final_User_Fact.csv' # 방금 만든 제출 파일

print("--- [Simple MAE Check] 1:1 행단위 비교 ---")

try:
    # 1. 파일 로드 (시간 파싱 안 함 -> 오직 값만 비교)
    df_true = pd.read_csv(TRUE_FILE)
    df_pred = pd.read_csv(SUB_FILE)

    # 2. 행 개수 검증 (이게 다르면 비교 불가)
    if len(df_true) != len(df_pred):
        print(f"[오류] 행 개수가 다릅니다! True: {len(df_true)}, Pred: {len(df_pred)}")
        print("파일이 잘못 생성되었거나, true.csv가 다른 파일입니다.")
    
    else:
        # 3. MAE 계산 (순서가 같다고 확신하므로 바로 계산)
        # nins 컬럼의 값만 쏙 뽑아서(values) 비교합니다. 인덱스나 시간 신경 안 씀.
        y_true = df_true['nins'].values
        y_pred = df_pred['nins'].values
        
        mae = mean_absolute_error(y_true, y_pred)
        
        print("\n" + "="*40)
        print(f" 📉 최종 MAE: {mae:.6f}")
        print("="*40)
        
        # 결과 해석
        if mae < 55:
            print(">> [성공] 정상 범위(53점대)입니다.")
        elif mae > 130:
            print(">> [경고] 점수가 여전히 130점대라면, UTC/KST 시차 문제입니다.")
            print(">> (0시=낮, 12시=밤으로 예측된 상황)")
        else:
            print(f">> [확인] 점수({mae})가 예상보다 높습니다. 보간법 차이일 수 있습니다.")

except KeyError:
    print("[오류] 파일에 'nins' 컬럼이 없습니다.")
except Exception as e:
    print(f"[오류] {e}")

--- [Simple MAE Check] 1:1 행단위 비교 ---

 📉 최종 MAE: 66.473928
>> [확인] 점수(66.47392760948647)가 예상보다 높습니다. 보간법 차이일 수 있습니다.


In [2]:
import pandas as pd
import numpy as np
import time
import warnings
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib

# [Configuration] 경고 메시지 출력 방지 (Clean Output)
warnings.filterwarnings('ignore')

print("--- [Final Hybrid Pipeline] Data-Driven Context + Physics-Guided Logic ---")
start_total = time.time()

# ==============================================================================
# [Step 0] 환경 설정 및 하이퍼파라미터
# ==============================================================================
CSV_TRAIN = 'train.csv'
CSV_TEST = 'test.csv'
SAMPLE_SUB = 'submission_sample.csv'
OUTPUT_FILE = 'submission_Final_Hybrid.csv'

# [Hyperparameters] 실험 기반 휴리스틱 튜닝 (Heuristic Tuning)
# 풍부한 시차(Lag) 변수 학습을 위해 모델 복잡도(num_leaves)를 유지하되,
# 과적합 방지를 위해 subsample(행 샘플링)과 colsample(열 샘플링) 적용
LGB_PARAMS = {
    'objective': 'mae',
    'metric': 'mae',
    'n_estimators': 3000,
    'learning_rate': 0.03,
    'num_leaves': 127,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'min_data_in_leaf': 50,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

# ==============================================================================
# [Step 1] 데이터 로드 및 통합 (One-Universe)
# ==============================================================================
print("\n[Step 1] 데이터 로드 및 무결성 확보...")

# 날짜 파싱을 포함한 고속 로드
df_train = pd.read_csv(CSV_TRAIN, parse_dates=['time'])
df_test = pd.read_csv(CSV_TEST, parse_dates=['time'])

# [Safety] 제출 시 순서 섞임 방지를 위한 원본 인덱스 백업
df_test['original_index'] = df_test.index

# 타겟(Target) 분리
y_train = df_train['nins'].copy()

# Data Leakage 방지를 위해 타겟 및 파생 컬럼 제거
drop_cols = ['nins', 'energy']
df_train.drop(columns=[c for c in drop_cols if c in df_train.columns], inplace=True)
df_test.drop(columns=[c for c in drop_cols if c in df_test.columns], inplace=True)

# 데이터셋 구분 태그
df_train['dataset_type'] = 'train'
df_test['dataset_type'] = 'test'

# [Strategy] One-Universe 통합 파이프라인
# Train/Test의 스케일링 및 군집화 기준 통일을 위해 병합 수행
df_full = pd.concat([df_train, df_test], ignore_index=True)
print(f" -> 통합 완료. 총 행 수: {len(df_full)}")


# ==============================================================================
# [Step 2] 결측치 처리 (하이브리드 보간)
# ==============================================================================
print("\n[Step 2] 하이브리드 보간(Hybrid Interpolation) 수행...")

cols_weather = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure', 
    'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 'temp_b', 'uv_idx', 'vis', 
    'wind_chill_temp', 'wind_dir_b', 'wind_gust_spd', 'wind_spd_b', 'cloud_a', 
    'ground_press', 'humidity', 'rain', 'snow', 'temp_a', 'temp_max', 'temp_min', 
    'wind_dir_a', 'wind_spd_a'
]
# 실제 존재하는 컬럼만 필터링
cols_exist = [c for c in cols_weather if c in df_full.columns]

# [Logic] 물리적 연속성(Continuity) 보존을 위해 발전소별 시간순 정렬
df_full.sort_values(by=['pv_id', 'time'], inplace=True)

# [Imputation] 시계열 추세를 끊지 않기 위해 Forward/Backward Fill 적용
df_full[cols_exist] = df_full.groupby('pv_id')[cols_exist].ffill().bfill()


# ==============================================================================
# [Step 3] 이중 군집화 (Dual Clustering)
# ==============================================================================
print("\n[Step 3] 이중 군집화 (Dual Clustering) 전략 적용...")
scaler = StandardScaler()

# 3-1. 위치 군집 (Location Cluster) - Micro Segmentation
# 지형적 특성(음영, 고도 등)을 반영하기 위해 K=20으로 세분화
loc_features = ['coord1', 'coord2']
scaled_loc = scaler.fit_transform(df_full[loc_features])
kmeans_loc = KMeans(n_clusters=20, random_state=42, n_init=10)
df_full['cluster_loc'] = kmeans_loc.fit_predict(scaled_loc)
df_full['cluster_loc'] = df_full['cluster_loc'].astype('category')

# 3-2. 기후 군집 (Climate Cluster) - Macro Segmentation
# 지역 전반의 거시적 기상 패턴을 반영하기 위해 K=10 설정
clim_features = ['temp_a', 'humidity', 'wind_spd_a', 'pressure', 'vis']
target_climate = [c for c in clim_features if c in df_full.columns]
pv_stats = df_full.groupby('pv_id')[target_climate].mean() # 발전소별 기후 프로필

scaled_cli = scaler.fit_transform(pv_stats)
kmeans_cli = KMeans(n_clusters=10, random_state=42, n_init=10)
pv_labels = kmeans_cli.fit_predict(scaled_cli)

# 매핑 (Mapping)
climate_map = dict(zip(pv_stats.index, pv_labels))
df_full['cluster_climate'] = df_full['pv_id'].map(climate_map).astype('category')


# ==============================================================================
# [Step 4] 특성 공학 (물리 공식 + Rich Context)
# ==============================================================================
print("\n[Step 4] 하이브리드 특성 공학 (Physics + Data)...")

# 4-1. 시간 변수 (Cyclical Encoding)
df_full['hour'] = df_full['time'].dt.hour
df_full['minute'] = df_full['time'].dt.minute
df_full['day_of_year'] = df_full['time'].dt.dayofyear

# 시간의 연속성을 위해 Sin/Cos 변환
df_full['hour_sin'] = np.sin(2 * np.pi * df_full['hour'] / 24.0)
df_full['hour_cos'] = np.cos(2 * np.pi * df_full['hour'] / 24.0)

# 4-2. [Physics] 태양 기하학(Solar Geometry) 시뮬레이션
# 날짜와 시간에 따른 이론적 태양 에너지 가용량(Daylight Cosine) 계산
# 가정: 한국 시간(KST) 기준 남중 시각(Solar Noon) 약 12:00
solar_noon = 12.0 
day_duration = 12.0 + 2.5 * np.sin(2 * np.pi * (df_full['day_of_year'] - 80) / 365.25)

df_full['decimal_hour'] = df_full['hour'] + df_full['minute'] / 60.0
scaled_time = (df_full['decimal_hour'] - solar_noon) * np.pi / (day_duration / 2)

# 'daylight_cosine'은 Step 7의 야간 마스킹(Night Masking)에 활용됨
df_full['daylight_cosine'] = np.cos(scaled_time) 

# 4-3. [Data-Driven] 대규모 시차(Lag) 변수 생성
# 모델에게 풍부한 문맥(Context)을 제공하기 위해 26개 모든 센서 변수의 과거 데이터 생성
print(" -> 모델 문맥 강화를 위한 Lag 변수 대량 생성 중...")

for col in cols_exist:
    # Lag 6 (30분 전), Lag 12 (1시간 전)
    df_full[f'{col}_lag30m'] = df_full.groupby('pv_id')[col].shift(6)
    df_full[f'{col}_lag1h'] = df_full.groupby('pv_id')[col].shift(12)

print(f" -> 특성 확장 완료. 총 변수 개수: {df_full.shape[1]}")

# Lag 생성으로 인한 초기 결측치는 Backfill로 처리
df_full = df_full.groupby('pv_id').bfill()


# ==============================================================================
# [Step 5] 데이터 분리 (Split)
# ==============================================================================
print("\n[Step 5] 학습/테스트 데이터 분리...")
mask_train = df_full['dataset_type'] == 'train'
mask_test = df_full['dataset_type'] == 'test'

X_train = df_full.loc[mask_train].copy()
X_test = df_full.loc[mask_test].copy()

# 학습 피처 선정 (수치형 + 범주형)
feats_num = list(X_train.select_dtypes(include=['number']).columns)
feats_cat = list(X_train.select_dtypes(include=['category']).columns)
features = feats_num + feats_cat

# 학습 방해 컬럼(메타 데이터) 제외
exclude_cols = {'nins', 'energy', 'pv_id', 'dataset_type', 'time', 'original_index'}
features = [f for f in features if f not in exclude_cols]
cat_feats = [f for f in feats_cat if f in features]

print(f" -> 최종 학습 변수 개수: {len(features)}")


# ==============================================================================
# [Step 6] 모델 학습 (LightGBM)
# ==============================================================================
print("\n[Step 6] LightGBM 모델 학습 시작...")
model = lgb.LGBMRegressor(**LGB_PARAMS)

model.fit(
    X_train[features], y_train,
    eval_set=[(X_train[features], y_train)],
    categorical_feature=cat_feats,
    callbacks=[
        lgb.early_stopping(stopping_rounds=100, verbose=False),
        lgb.log_evaluation(period=1000)
    ]
)


# ==============================================================================
# [Step 7] 예측 및 물리적 후처리 (Safety Mechanisms)
# ==============================================================================
print("\n[Step 7] 추론 및 안전장치 적용...")
preds = model.predict(X_test[features])

# 7-1. 기본 보정 (음수 발전량 제거)
preds = np.maximum(preds, 0)

# 7-2. [Physics] 야간 마스킹 (Daylight Masking)
# 'daylight_cosine' 값이 0 이하(해짐)인 경우 강제로 0 처리하여 물리적 정합성 확보
if 'daylight_cosine' in X_test.columns:
    # 부동소수점 오차 고려한 임계값 설정
    mask_night = X_test['daylight_cosine'] <= 0.0001
    preds[mask_night] = 0
    print(f" -> [후처리] 물리적 밤 시간대 {sum(mask_night)}건을 0으로 보정했습니다.")

# 7-3. [Safety] 인덱스 복원 (Index Restoration)
# 제출 파일의 순서가 원본 test.csv와 완벽하게 일치하도록 재정렬
results = pd.DataFrame({
    'nins': preds,
    'original_index': X_test['original_index'].values
})

results.sort_values(by='original_index', inplace=True)

# ==============================================================================
# [Step 8] 제출 파일 생성
# ==============================================================================
sub = pd.read_csv(SAMPLE_SUB)
sub['nins'] = results['nins'].values
sub.to_csv(OUTPUT_FILE, index=False)

print(f"\n[성공] 최종 제출 파일이 '{OUTPUT_FILE}'로 저장되었습니다.")
print(f"총 소요 시간: {time.time() - start_total:.2f} 초")

--- [Final Hybrid Pipeline] Data-Driven Context + Physics-Guided Logic ---

[Step 1] 데이터 로드 및 무결성 확보...
 -> 통합 완료. 총 행 수: 22075188

[Step 2] 하이브리드 보간(Hybrid Interpolation) 수행...

[Step 3] 이중 군집화 (Dual Clustering) 전략 적용...

[Step 4] 하이브리드 특성 공학 (Physics + Data)...
 -> 모델 문맥 강화를 위한 Lag 변수 대량 생성 중...
 -> 특성 확장 완료. 총 변수 개수: 94

[Step 5] 학습/테스트 데이터 분리...


MemoryError: Unable to allocate 12.2 GiB for an array with shape (85, 19236948) and data type float64